In [1]:
import numpy as np
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math
import random

MIN_LIST_LEN = 6
MAX_LIST_LEN = 6
MAX_STEPS = 120

SUCCESS_REWARD = 2.0
STEP_REWARD = -0.1
SWAP_REWARD = 1.0
INVALID_ACTION_REWARD = -10.0

EPS_START = 0.5
EPS_END = 0.05
EPS_DECAY = 1000
GAMMA = 0.8
NUM_EPISODES = 100000
EPISODES_SAVE = 1000
OUTPUT_DIR = 'datasets/rl_sort_transformer_easy/list6_transformer3_128_gamma098_step120_v1'

# Define the vocabulary
vocab = {
    'Comparison': 0,
    'Swap': 1,
    '0': 2,
    '1': 3,
    '2': 4,
    '3': 5,
    '4': 6,
    '5': 7,
    '6': 8,
    '7': 9,
    'less': 10,
    'equal': 11,
    'more': 12,
    'len1': 13,
    'len2': 14,
    'len3': 15,
    'len4': 16,
    'len5': 17,
    'len6': 18,
    'len7': 19,
    'len8': 20,
}
inv_vocab = {v: k for k, v in vocab.items()}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the environment
class SortingEnv:
    def __init__(self):
        self.max_steps = MAX_STEPS

    def reset(self):
        self.length = random.randint(MIN_LIST_LEN, MAX_LIST_LEN)
        self.list = [random.randint(1, 100) for _ in range(self.length)]
        while self.list == sorted(self.list):
            self.list = [random.randint(1, 100) for _ in range(self.length)]
        self.indices = None
        self.current_step = 0
        self.done = False
        initial_token = 'len{}'.format(self.length)
        return vocab[initial_token], self.list.copy()
    
    def get_list(self):
        return self.list
    
    def get_list_len(self):
        return len(self.list)

    def step(self, action_tokens):
        action = action_tokens[0]
        reward = -0.01  # default penalty
        response_token = None

        if action == vocab['Comparison']:
            if len(action_tokens) != 3:
                reward = INVALID_ACTION_REWARD
                self.done = True
                return response_token, reward, self.done, self.list.copy()
            index1 = action_tokens[1] - vocab['0']
            index2 = action_tokens[2] - vocab['0']
            if index1 >= self.length or index2 >= self.length or index1 < 0 or index2 < 0:
                reward = INVALID_ACTION_REWARD
                self.done = True
                return response_token, reward, self.done, self.list.copy()
            self.indices = (index1, index2)
            if self.list[index1] < self.list[index2]:
                response_token = vocab['less']
                reward = STEP_REWARD
            elif self.list[index1] == self.list[index2]:
                response_token = vocab['equal']
                reward = STEP_REWARD * 2
            else:
                response_token = vocab['more']
                reward = STEP_REWARD
        elif action == vocab['Swap']:
            if self.indices is None:
                reward = INVALID_ACTION_REWARD
                self.done = True
                return response_token, reward, self.done, self.list.copy()
            index1, index2 = self.indices
            prev_list = self.list.copy()
            self.list[index1], self.list[index2] = self.list[index2], self.list[index1]
            if self.list == sorted(self.list):
                reward = SUCCESS_REWARD
                self.done = True
            #elif prev_list[index1] > prev_list[index2] and self.list[index1] <= self.list[index2]:
            #    reward = 0.1
            elif (index1 < index2 and prev_list[index1] > prev_list[index2] and self.list[index1] <= self.list[index2]) or \
                (index1 > index2 and prev_list[index1] < prev_list[index2] and self.list[index1] >= self.list[index2]):
                reward = SWAP_REWARD
            elif (index1 < index2 and prev_list[index1] < prev_list[index2] and self.list[index1] >= self.list[index2]) or \
                (index1 > index2 and prev_list[index1] > prev_list[index2] and self.list[index1] <= self.list[index2]):
                reward = -SWAP_REWARD
            else:
                reward = STEP_REWARD
            self.indices = None
        else:
            reward = INVALID_ACTION_REWARD
            self.done = True

        self.current_step += 1
        if self.current_step >= self.max_steps:
            self.done = True
        return response_token, reward, self.done, self.list.copy()


Using device: cuda


In [2]:
# Positional Encoding for Transformer
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=256):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=0.1)

        pe = torch.zeros(max_len, d_model)  # (max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)  # (max_len, 1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() *
                             (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)  # Even indices
        pe[:, 1::2] = torch.cos(position * div_term)  # Odd indices
        pe = pe.unsqueeze(1)  # (max_len, 1, d_model)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

# Transformer Model
class TransformerModel(nn.Module):
    def __init__(self, vocab_size, d_model=128, nhead=8, num_layers=3):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.d_model = d_model

        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.decoder = nn.Linear(d_model, vocab_size)

        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.embedding.weight, -initrange, initrange)
        nn.init.zeros_(self.decoder.bias)
        nn.init.uniform_(self.decoder.weight, -initrange, initrange)

    def forward(self, src):
        src = self.embedding(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
        output = self.decoder(output)
        return output

def decode(input_tokens, inv_vocab):
    return ' '.join([inv_vocab[x] for x in input_tokens])


def save_checkpoint(model, optimizer, episode, folder, filename):
    """
    Save the model and optimizer state to the designated filepath.

    Args:
        model (nn.Module): The model to save.
        optimizer (torch.optim.Optimizer): The optimizer whose state to save.
        episode (int): The current episode number.
        filepath (str): The path where to save the checkpoint.
    """
    filepath = os.path.join(folder, filename)
    # Ensure the directory exists
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    # Save the checkpoint
    torch.save({
        'episode': episode,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, filepath)
    print(f"Checkpoint saved at episode {episode} to {filepath}")

def load_checkpoint(filepath, model, optimizer):
    """
    Load the model and optimizer state from the designated filepath.

    Args:
        filepath (str): The path from where to load the checkpoint.
        model (nn.Module): The model into which to load the state_dict.
        optimizer (torch.optim.Optimizer): The optimizer into which to load the state.

    Returns:
        int: The episode number to resume from.
    """
    if os.path.isfile(filepath):
        checkpoint = torch.load(filepath, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        episode = checkpoint['episode']
        print(f"Checkpoint loaded from {filepath}, resuming from episode {episode}")
        return episode
    else:
        print(f"No checkpoint found at {filepath}, starting from scratch.")
        return 0

In [ ]:
# Training Loop
def train(verbose=False):
    # Removed torch.autograd.set_detect_anomaly(True)
    vocab_size = len(vocab)
    model = TransformerModel(vocab_size).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)  # Reduced learning rate
    # Optionally, load a checkpoint
    # load_checkpoint("path_to_checkpoint", model, optimizer)

    episode_cnt = 0
    total_reward = 0.0
    num_successes = 0
    total_stesp = 0
    
    for episode in range(NUM_EPISODES):
        t1 = time.time()
        model.train()  # Set model to training mode
        env = SortingEnv()
        initial_token_id, current_list = env.reset()
        input_tokens = [initial_token_id]
        log_probs = []
        rewards = []
        
        state = 'expect_action'
        done = False
        success = False

        while not done and len(input_tokens) < env.max_steps:
            if verbose:
                print(decode(input_tokens, inv_vocab))
                print(env.get_list())
            # Prepare input tensor
            input_seq = torch.tensor(input_tokens, dtype=torch.long, device=device).unsqueeze(1)  # (seq_len, batch_size)
            # Get model output
            with torch.autocast(device_type=device.type, dtype=torch.bfloat16):
                output = model(input_seq)  # (seq_len, batch_size, vocab_size)
                # Get logits for the last token
                logits = output[-1, 0, :]  # (vocab_size)

                # Check for NaNs in logits
                if torch.isnan(logits).any():
                    print(f"Episode {episode}, NaNs in logits before masking.")
                    break

                # Get valid tokens based on state
                def get_valid_tokens(state):
                    action_tokens = [vocab['Comparison'], vocab['Swap']]
                    index_tokens = [vocab[str(i)] for i in range(env.length)]
                    if state == 'expect_action':
                        return action_tokens
                    elif state == 'expect_index1':
                        return index_tokens[:-1]
                    elif state == 'expect_index2':
                        return [x for x in index_tokens if x > input_tokens[-1]]
                    else:
                        # Handle unexpected states by defaulting to expect_action
                        return action_tokens

                valid_token_ids = get_valid_tokens(state)

                # Ensure valid_token_ids are within the vocab range
                if any(idx >= vocab_size or idx < 0 for idx in valid_token_ids):
                    print(f"Episode {episode}, invalid indices in valid_token_ids: {valid_token_ids}")
                    break

                # Mask invalid tokens
                mask_value = -1e9  # Use a large negative value instead of -inf
                mask = torch.full_like(logits, mask_value).to(device)
                mask[valid_token_ids] = 0
                masked_logits = logits + mask

                # Sample action. Have some chance to randomly pick a valid action.
                eps_threshold = EPS_END + (EPS_START - EPS_END) * np.exp(-1.0 * episode / EPS_DECAY)
                if random.random() < eps_threshold:
                    masked_logits = masked_logits / 4

                # Check for NaNs in masked_logits
                if torch.isnan(masked_logits).any():
                    print(f"Episode {episode}, NaNs in masked_logits after masking.")
                    break

                # Compute probabilities
                probs = F.softmax(masked_logits, dim=0)

                # Check for NaNs in probs
                if torch.isnan(probs).any():
                    print(f"Episode {episode}, NaNs in probs after softmax.")
                    break

                try:
                    m = torch.distributions.Categorical(probs)
                    action_token = m.sample()
                    log_prob = m.log_prob(action_token)
                except ValueError as e:
                    print(f"Episode {episode}, error in sampling action: {e}")
                    break

            log_probs.append(log_prob)
            input_tokens.append(action_token.item())

            action = action_token.item()
            reward = 0.0
            if state == 'expect_action':
                if action == vocab['Comparison']:
                    state = 'expect_index1'
                elif action == vocab['Swap']:
                    if env.indices is None:
                        reward = INVALID_ACTION_REWARD
                        rewards.append(reward)
                        done = True
                        continue
                    action_tokens = [vocab['Swap']]
                    response_token, reward, done, current_list = env.step(action_tokens)
                    if done and reward == SUCCESS_REWARD:
                        success = True
                    if verbose:
                        print("Reward:", reward)
                    state = 'expect_action'
                else:
                    reward = INVALID_ACTION_REWARD
                    done = True
            elif state == 'expect_index1':
                index1_token = action_token
                state = 'expect_index2'
            elif state == 'expect_index2':
                index2_token = action_token
                action_tokens = [vocab['Comparison'], index1_token.item(), index2_token.item()]
                response_token, reward, done, current_list = env.step(action_tokens)
                if done and reward == SUCCESS_REWARD:
                    success = True
                if verbose:
                    print("Reward:", reward)
                if response_token is not None:
                    input_tokens.append(response_token)
                state = 'expect_action'
            else:
                reward = INVALID_ACTION_REWARD
                done = True

            rewards.append(reward)
        #
        if success: 
            num_successes += 1
        # Save checkpoint
        if episode > 0 and episode % EPISODES_SAVE == 0:
            avg_reward = total_reward / episode_cnt
            success_rate = num_successes / episode_cnt
            avg_steps = total_steps / episode_cnt
            episode_cnt = 0
            total_reward = 0.0
            num_successes = 0
            total_steps = 0
            save_checkpoint(model, optimizer, episode, OUTPUT_DIR, f"ckpt_{episode}_{success_rate:.4f}_{avg_steps:.2f}.pth")
        #
        assert len(log_probs) == len(rewards), "log_probs and returns have different sizes!"

        if len(log_probs) == 0:
            continue  # Skip if no actions were taken

        # Compute returns and loss within autocast
        with torch.autocast(device_type=device.type, dtype=torch.bfloat16):
            # Compute returns
            returns = []
            R = 0
            gamma = GAMMA
            for r in rewards[::-1]:
                R = r + gamma * R
                returns.insert(0, R)
            returns = torch.tensor(returns).to(device)

            # Check for NaNs in returns
            if torch.isnan(returns).any():
                print(f"Episode {episode}, NaNs in returns.")
                continue

            # Compute loss
            loss = 0
            for log_prob, R in zip(log_probs, returns):
                loss -= log_prob * R

            # Check for NaNs in loss
            if torch.isnan(loss):
                print(f"Episode {episode}, NaN in loss.")
                continue

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
        optimizer.step()

        episode_cnt += 1
        total_reward += sum(rewards)
        total_steps += len(rewards)
        t2 = time.time()
        if episode % 1 == 0:
            print(f"Episode {episode}, loss:{loss.item():.4f}, {'succeed' if success else 'fail'}, steps:{len(rewards)}, total reward:{sum(rewards):.4f}, {t2-t1} sec")

if __name__ == "__main__":
    train(verbose=False)


/home/mcwave/.local/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Episode 0, loss:-56.5354, fail, steps:26, total reward:-11.8000, 0.4371922016143799 sec
Episode 1, loss:-7.0741, fail, steps:1, total reward:-10.0000, 0.005114078521728516 sec
Episode 2, loss:-59.6471, fail, steps:26, total reward:-12.7000, 0.05188727378845215 sec
Episode 3, loss:-47.3797, fail, steps:9, total reward:-10.2000, 0.018344640731811523 sec
Episode 4, loss:-49.1862, fail, steps:34, total reward:-10.0000, 0.06811308860778809 sec
Episode 5, loss:-8.9220, fail, steps:1, total reward:-10.0000, 0.003808259963989258 sec
Episode 6, loss:-49.8247, fail, steps:17, total reward:-11.5000, 0.03262758255004883 sec
Episode 7, loss:-8.7725, fail, steps:91, total reward:-1.8000, 0.20496225357055664 sec
Episode 8, loss:-19.1759, fail, steps:90, total reward:-3.9000, 0.18864917755126953 sec
Episode 9, loss:-13.5208, fail, steps:1, total reward:-10.0000, 0.007322072982788086 sec
Episode 10, loss:-12.0954, fail, steps:90, total reward:-2.9000, 0.17507171630859375 sec
Episode 11, loss:-11.3962, 

Episode 94, loss:-55.0067, fail, steps:60, total reward:-5.7000, 0.125762939453125 sec
Episode 95, loss:-23.8561, fail, steps:1, total reward:-10.0000, 0.006243705749511719 sec
Episode 96, loss:6.9020, succeed, steps:82, total reward:3.5000, 0.1584022045135498 sec
Episode 97, loss:-2.7751, fail, steps:92, total reward:1.2000, 0.18661880493164062 sec
Episode 98, loss:10.4001, fail, steps:92, total reward:3.2000, 0.1897110939025879 sec
Episode 99, loss:-81.3177, fail, steps:43, total reward:-10.3000, 0.09238481521606445 sec
Episode 100, loss:3.8695, fail, steps:91, total reward:3.2000, 0.18343639373779297 sec
Episode 101, loss:-41.9504, fail, steps:48, total reward:-6.4000, 0.10259199142456055 sec
Episode 102, loss:3.3006, fail, steps:91, total reward:2.2000, 0.18774700164794922 sec
Episode 103, loss:0.0317, fail, steps:92, total reward:2.3000, 0.190887451171875 sec
Episode 104, loss:10.0640, succeed, steps:72, total reward:4.7000, 0.15116429328918457 sec
Episode 105, loss:0.7365, fail, 

Episode 187, loss:-5.5013, fail, steps:90, total reward:0.1000, 0.22359466552734375 sec
Episode 188, loss:13.3439, succeed, steps:84, total reward:5.5000, 0.17084598541259766 sec
Episode 189, loss:24.2125, succeed, steps:73, total reward:8.9000, 0.15061402320861816 sec
Episode 190, loss:18.1315, succeed, steps:86, total reward:6.4000, 0.17567920684814453 sec
Episode 191, loss:8.0054, succeed, steps:55, total reward:3.3000, 0.11604475975036621 sec
Episode 192, loss:6.5158, succeed, steps:54, total reward:2.3000, 0.11428213119506836 sec
Episode 193, loss:5.7710, succeed, steps:30, total reward:3.1000, 0.062325477600097656 sec
Episode 194, loss:11.3322, succeed, steps:56, total reward:4.3000, 0.11161494255065918 sec
Episode 195, loss:-47.7976, fail, steps:5, total reward:-9.1000, 0.013273477554321289 sec
Episode 196, loss:12.6555, succeed, steps:33, total reward:6.1000, 0.06359076499938965 sec
Episode 197, loss:12.2167, fail, steps:92, total reward:5.3000, 0.1831493377685547 sec
Episode 1

Episode 279, loss:-1.6763, fail, steps:91, total reward:1.2000, 0.18364310264587402 sec
Episode 280, loss:16.6226, succeed, steps:60, total reward:6.3000, 0.12438821792602539 sec
Episode 281, loss:-49.7493, fail, steps:22, total reward:-7.6000, 0.047635555267333984 sec
Episode 282, loss:-0.4283, fail, steps:91, total reward:1.2000, 0.18080615997314453 sec
Episode 283, loss:8.4619, fail, steps:92, total reward:4.3000, 0.18828845024108887 sec
Episode 284, loss:4.2992, succeed, steps:11, total reward:2.7000, 0.028060436248779297 sec
Episode 285, loss:12.7682, succeed, steps:76, total reward:5.7000, 0.14895033836364746 sec
Episode 286, loss:5.1742, fail, steps:91, total reward:2.2000, 0.18470191955566406 sec
Episode 287, loss:5.8176, succeed, steps:42, total reward:2.7000, 0.08843111991882324 sec
Episode 288, loss:10.4529, fail, steps:91, total reward:4.2000, 0.1829066276550293 sec
Episode 289, loss:-39.9328, fail, steps:5, total reward:-9.1000, 0.015851259231567383 sec
Episode 290, loss:1

...

Episode 3677, loss:0.0016, succeed, steps:36, total reward:2.8000, 0.07343602180480957 sec
Episode 3678, loss:6.3664, succeed, steps:87, total reward:7.4000, 0.1722729206085205 sec
Episode 3679, loss:0.4637, succeed, steps:49, total reward:6.6000, 0.10179662704467773 sec
Episode 3680, loss:0.3902, succeed, steps:37, total reward:7.0000, 0.07545232772827148 sec
Episode 3681, loss:-0.0375, fail, steps:91, total reward:2.2000, 0.1831064224243164 sec
Episode 3682, loss:1.8845, succeed, steps:77, total reward:9.8000, 0.15955138206481934 sec
Episode 3683, loss:5.6643, succeed, steps:40, total reward:6.9000, 0.08374977111816406 sec
Episode 3684, loss:0.0885, fail, steps:91, total reward:4.2000, 0.1817033290863037 sec
Episode 3685, loss:1.0803, succeed, steps:75, total reward:7.8000, 0.15526533126831055 sec
Episode 3686, loss:0.5576, succeed, steps:44, total reward:7.8000, 0.0913999080657959 sec
Episode 3687, loss:0.0046, succeed, steps:22, total reward:4.4000, 0.04607677459716797 sec
Episode 

Episode 3769, loss:0.2740, succeed, steps:36, total reward:6.0000, 0.07221460342407227 sec
Episode 3770, loss:1.7875, succeed, steps:18, total reward:3.5000, 0.03718829154968262 sec
Episode 3771, loss:0.1520, succeed, steps:43, total reward:6.8000, 0.08314657211303711 sec
Episode 3772, loss:0.4918, succeed, steps:45, total reward:8.8000, 0.08919906616210938 sec
Episode 3773, loss:0.2563, succeed, steps:73, total reward:8.9000, 0.14479732513427734 sec
Episode 3774, loss:1.2457, succeed, steps:40, total reward:6.9000, 0.08240771293640137 sec
Episode 3775, loss:0.5271, succeed, steps:39, total reward:5.9000, 0.07897424697875977 sec
Episode 3776, loss:0.2181, succeed, steps:52, total reward:9.6000, 0.10260581970214844 sec
Episode 3777, loss:0.1876, succeed, steps:37, total reward:6.9000, 0.07527780532836914 sec
Episode 3778, loss:0.0005, succeed, steps:25, total reward:4.2000, 0.050513267517089844 sec
Episode 3779, loss:1.4978, succeed, steps:68, total reward:10.1000, 0.13265609741210938 s

Episode 3860, loss:0.0115, succeed, steps:34, total reward:7.1000, 0.06757068634033203 sec
Episode 3861, loss:2.7087, succeed, steps:63, total reward:11.3000, 0.12437844276428223 sec
Episode 3862, loss:0.1060, succeed, steps:36, total reward:6.0000, 0.07306194305419922 sec
Episode 3863, loss:-0.4162, fail, steps:92, total reward:6.0000, 0.18298053741455078 sec
Episode 3864, loss:0.1987, succeed, steps:33, total reward:6.1000, 0.0702199935913086 sec
Episode 3865, loss:1.8348, succeed, steps:34, total reward:7.1000, 0.06810903549194336 sec
Episode 3866, loss:0.2334, succeed, steps:55, total reward:9.5000, 0.1088719367980957 sec
Episode 3867, loss:0.8228, succeed, steps:44, total reward:7.8000, 0.08847904205322266 sec
Episode 3868, loss:1.1730, succeed, steps:70, total reward:5.9000, 0.13976144790649414 sec
Episode 3869, loss:0.6291, succeed, steps:38, total reward:8.0000, 0.07833123207092285 sec
Episode 3870, loss:1.0721, succeed, steps:33, total reward:6.1000, 0.06653428077697754 sec
Ep

Episode 3952, loss:1.1308, succeed, steps:85, total reward:11.2000, 0.1694941520690918 sec
Episode 3953, loss:0.0010, succeed, steps:75, total reward:4.7000, 0.15420174598693848 sec
Episode 3954, loss:-0.1368, fail, steps:91, total reward:3.2000, 0.18852996826171875 sec
Episode 3955, loss:1.2644, succeed, steps:51, total reward:8.6000, 0.1066286563873291 sec
Episode 3956, loss:-0.0333, succeed, steps:69, total reward:4.8000, 0.1432936191558838 sec
Episode 3957, loss:1.6413, fail, steps:92, total reward:6.3000, 0.1902623176574707 sec
Episode 3958, loss:1.1028, succeed, steps:61, total reward:9.3000, 0.12848997116088867 sec
Episode 3959, loss:0.6044, succeed, steps:44, total reward:7.8000, 0.0925588607788086 sec
Episode 3960, loss:-0.0014, succeed, steps:86, total reward:3.3000, 0.1742253303527832 sec
Episode 3961, loss:0.9283, succeed, steps:61, total reward:9.3000, 0.12766265869140625 sec
Episode 3962, loss:0.0008, succeed, steps:25, total reward:4.2000, 0.05293440818786621 sec
Episode

Episode 4042, loss:0.3681, succeed, steps:85, total reward:11.6000, 0.1768476963043213 sec
Episode 4043, loss:0.0372, succeed, steps:33, total reward:6.1000, 0.07088184356689453 sec
Episode 4044, loss:0.0943, succeed, steps:38, total reward:8.0000, 0.07725119590759277 sec
Episode 4045, loss:0.5152, succeed, steps:50, total reward:7.6000, 0.09942913055419922 sec
Episode 4046, loss:3.8997, succeed, steps:64, total reward:6.1000, 0.12707924842834473 sec
Episode 4047, loss:0.0067, succeed, steps:15, total reward:3.6000, 0.033327579498291016 sec
Episode 4048, loss:0.0004, succeed, steps:26, total reward:5.3000, 0.051354169845581055 sec
Episode 4049, loss:0.0296, succeed, steps:38, total reward:8.0000, 0.07392644882202148 sec
Episode 4050, loss:0.0490, succeed, steps:42, total reward:5.8000, 0.08231210708618164 sec
Episode 4051, loss:0.1942, succeed, steps:28, total reward:4.2000, 0.05644965171813965 sec
Episode 4052, loss:2.4492, succeed, steps:36, total reward:6.0000, 0.07084441184997559 s